# MyBinder Demo with MNIST

Here's a demo showing what mybinder can do. Tensorflow, the jupyter extensions, all of it are installed via the included docker file. 

Below I build a non-trivial CNN network that trains quite fast and ends up with near [state-of-the-art](http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html#4d4e495354) results

In [1]:
%pylab inline
import keras
from keras import backend as K
import tensorflow as tf

Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = (x_train - np.mean(x_train, axis = (1,2), keepdims=True))/np.std(x_train, axis = (1,2), keepdims=True)
x_test = (x_test - np.mean(x_test, axis = (1,2), keepdims=True))/np.std(x_test, axis = (1,2), keepdims=True)


11493376/11490434 [==============================] - 2s 0us/step


In [3]:
callbacks = [
    keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=1, verbose=1),
    keras.callbacks.EarlyStopping(patience=4)
]

In [11]:
fc_model = keras.Sequential([
    keras.layers.Reshape((28,28,1), input_shape=x_train.shape[1:]),
    
    keras.layers.Conv2D(10,5,activation="elu", padding="same"),
    keras.layers.MaxPool2D(3,2, padding="same"),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(15,4,activation="elu", padding="same"),
    keras.layers.MaxPool2D(3,2, padding="same"),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(20,3,activation="elu", padding="same"),
    keras.layers.MaxPool2D(3,2, padding="same"),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(25,3,activation="elu", padding="same"),
    keras.layers.MaxPool2D(3,2, padding="same"),
    keras.layers.BatchNormalization(),
    
    keras.layers.Flatten(),
    keras.layers.Dense(20, activation="elu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(np.max(y_train) + 1, activation="softmax")
])

fc_model.summary()

fc_model.compile(
    "nadam",
    "sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"])

fc_model.fit(
    x=x_train, y=y_train, validation_split=.2, epochs=100, callbacks=callbacks)

loss, acc = fc_model.evaluate(x_test, y_test)
print(f"validation accuracy: {loss:.2%}, loss {acc:.4f}")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_5 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 28, 28, 10)        260       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 14, 14, 10)        0         
_________________________________________________________________
batch_normalization_18 (Batc (None, 14, 14, 10)        40        
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 14, 14, 15)        2415      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 7, 7, 15)          0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 7, 7, 15)          60        
__________